In [57]:
# General Python
import sys
import os
sys.path.append('./')


# Data structures
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
dpi = 200
mpl.rc("savefig", dpi=dpi)
%matplotlib inline
from scipy.stats.stats import pearsonr
from scipy.stats.stats import spearmanr
from scipy import stats
from scipy.optimize import curve_fit
from scipy.optimize import leastsq
import seaborn as sns
import math
import re

from lmfit import minimize, Parameters, report_fit
from matplotlib.colors import LogNorm

#additional imports
import pandas as pd
from Bio import SeqIO, SeqRecord,Seq
import mkl_random
import random

import os
import subprocess
from Bio import Entrez, SeqIO
from copy import deepcopy
from pathlib import Path

import datetime
import time
import json
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

import seaborn as sb

import re


In [58]:
#define a function to return amino acid if given a codon sequence (single amino acid)
def get_amino_acid(codon_seq):
    from Bio.Seq import Seq
    messenger_rna = Seq(codon_seq)
    translation=list(messenger_rna.translate())
    amino_acid=translation[0]
    return (amino_acid)

#define a function to translate any DNA sequence
def get_translation(codon_seq):
    from Bio.Seq import Seq
    messenger_rna = Seq(codon_seq)
    translation=list(messenger_rna.translate())
    return (translation)

## Define plotting functions

In [59]:
#input desired subsetted mutagenesis window, plate number as ID, and subsetted dataframe
#return a histogram representing counts of mutants at each position within the window
#and among categories of variants outside the window/not single AA changes
def makeVariantHist(windowNum, plateNum, inputDF):
    #assign mutational region based on input to imports above (where windowNum is specified)
    residue_start = mutationD[windowNum][0]
    residue_end=mutationD[windowNum][1]

    #make list used for bar chart labels, these are categories that will be grouped from 
    #multiple residue positions
    variant_list=['WT', 'indel','5-UTR','outside']

    #create list of individual AA positions that will be counted for this plot
    #list of AA residues in window
    aa_list= list(mutationD[windowNum][2])
    #list of numerical positions of each residue
    aa_num_list=range(residue_start, residue_end+1)
    #concatenated item by item to give list of aa+pos for all residues in window
    res_list= [i + str(j) for i,j in zip (aa_list, aa_num_list)]

    #extend grouped residue categories list with individual AAs
    variant_list.extend(res_list)

    #create new dataframe and add contents of residue/grouped residue containing list
    plotDF=pd.DataFrame()
    plotDF['Genotype']=variant_list

    #make list of all mutated sites
    single_sub_sites=[]

    #pick out positions from substitution dataframe
    for index, val in inputDF.iterrows():
        sub=str(val.amino_acid_sub)
        codon_num=val.codon

        if codon_num==0:
            single_sub_sites.append("WT")
        if codon_num== -2:
            single_sub_sites.append("indel")
        if codon_num== -1:
            single_sub_sites.append("5-UTR")

        if "_" in sub:
            sub_list=sub.split('_')
            aa_num_corrected=int(sub_list[1])
            ref_pos=str(sub_list[0])+str(aa_num_corrected)

            if ((aa_num_corrected > residue_end) or (aa_num_corrected < residue_start)) and aa_num_corrected >-1:
                single_sub_sites.append("outside")

            else:
                single_sub_sites.append(ref_pos)

    #count occurences of each and add to dataframe containing reference seq
    count=[]
    for index, val in plotDF.iterrows():
        num_elements=single_sub_sites.count(val.Genotype)
        count.append(num_elements)

    plotDF['count']=count
    
    #make fig and ax
    fig=plt.figure(figsize=(20,10))
    ax=plt.gca()
    
    #make bar plot using genotype and count data
    ax.bar(list(plotDF['Genotype']),list(plotDF['count']), color='dodgerblue')
    
    #make string for title
    title="Plate " + str(plateNum) + ": Count of SpAP WT, indel and point mutants (n = "+str(len(inputDF))+" variants from "+str(dataDF['i7_barcode'].nunique())+" barcodes)"

    plt.ylabel('count',fontsize=24)
    plt.yticks(np.arange(0, plotDF['count'].max(), 5), fontsize=16)
    plt.xlabel('Residue (SpAP window '+str(windowNum)+", " +str(residue_start)+"-"+str(residue_end)+')',fontsize=24)
    plt.xticks(rotation=45, fontsize=16)
    plt.title(title, fontsize=24)
    plt.tight_layout()
    plt.close()
    return fig

In [60]:
## same function as above except return DF containing plotting data only, no fig
#input desired subsetted mutagenesis window, plate number as ID, and subsetted dataframe
#return a histogram representing counts of mutants at each position within the window
#and among categories of variants outside the window/not single AA changes
def makeCountDF(windowNum, plateNum, inputDF):
    #assign mutational region based on input to imports above (where windowNum is specified)
    residue_start = mutationD[windowNum][0]
    residue_end=mutationD[windowNum][1]

    #make list used for bar chart labels, these are categories that will be grouped from 
    #multiple residue positions
    variant_list=['WT', 'indel','5-UTR','outside']

    #create list of individual AA positions that will be counted for this plot
    #list of AA residues in window
    aa_list= list(mutationD[windowNum][2])
    #list of numerical positions of each residue
    aa_num_list=range(residue_start, residue_end+1)
    #concatenated item by item to give list of aa+pos for all residues in window
    res_list= [i + str(j) for i,j in zip (aa_list, aa_num_list)]

    #extend grouped residue categories list with individual AAs
    variant_list.extend(res_list)

    #create new dataframe and add contents of residue/grouped residue containing list
    plotDF=pd.DataFrame()
    plotDF['Genotype']=variant_list

    #make list of all mutated sites
    single_sub_sites=[]

    #pick out positions from substitution dataframe
    for index, val in inputDF.iterrows():
        sub=str(val.amino_acid_sub)
        codon_num=val.codon

        if codon_num==0:
            single_sub_sites.append("WT")
        if codon_num== -2:
            single_sub_sites.append("indel")
        if codon_num== -1:
            single_sub_sites.append("5-UTR")

        if "_" in sub:
            sub_list=sub.split('_')
            aa_num_corrected=int(sub_list[1])
            ref_pos=str(sub_list[0])+str(aa_num_corrected)

            if ((aa_num_corrected > residue_end) or (aa_num_corrected < residue_start)) and aa_num_corrected >-1:
                single_sub_sites.append("outside")

            else:
                single_sub_sites.append(ref_pos)

    #count occurences of each and add to dataframe containing reference seq
    count=[]
    for index, val in plotDF.iterrows():
        num_elements=single_sub_sites.count(val.Genotype)
        count.append(num_elements)

    plotDF['count']=count
    
    return plotDF

In [61]:
#input desired subsetted mutagenesis window, plate number as ID, and subsetted dataframe
#return a histogram representing counts of mutants at each position within the window
#and among categories of variants outside the window/not single AA changes
def makeVariantHist_full(inputDF):
    #assign mutational region based on input to imports above (where windowNum is specified)
    residue_start = 1
    residue_end=542

    #make list used for bar chart labels, these are categories that will be grouped from 
    #multiple residue positions
    variant_list=['WT', 'indel','5-UTR','outside']

    #create list of individual AA positions that will be counted for this plot
    #list of AA residues in window
    aa_list= protein
    #list of numerical positions of each residue
    aa_num_list=range(residue_start, residue_end+1)
    #concatenated item by item to give list of aa+pos for all residues in window
    res_list= [i + str(j) for i,j in zip (aa_list, aa_num_list)]

    #extend grouped residue categories list with individual AAs
    variant_list.extend(res_list)

    #create new dataframe and add contents of residue/grouped residue containing list
    plotDF=pd.DataFrame()
    plotDF['Genotype']=variant_list

    #make list of all mutated sites
    single_sub_sites=[]

    #pick out positions from substitution dataframe
    for index, val in inputDF.iterrows():
        sub=str(val.amino_acid_sub)
        codon_num=val.codon

        if codon_num==0:
            single_sub_sites.append("WT")
        if codon_num== -2:
            single_sub_sites.append("indel")
        if codon_num== -1:
            single_sub_sites.append("5-UTR")

        if "_" in sub:
            sub_list=sub.split('_')
            aa_num_corrected=int(sub_list[1])
            ref_pos=str(sub_list[0])+str(aa_num_corrected)

            if ((aa_num_corrected > residue_end) or (aa_num_corrected < residue_start)) and aa_num_corrected >-1:
                single_sub_sites.append("outside")

            else:
                single_sub_sites.append(ref_pos)

    #count occurences of each and add to dataframe containing reference seq
    count=[]
    for index, val in plotDF.iterrows():
        num_elements=single_sub_sites.count(val.Genotype)
        count.append(num_elements)

    plotDF['count']=count
    
    plotDF = plotDF[plotDF.Genotype != 'WT']
    plotDF = plotDF[plotDF.Genotype != 'indel']
    plotDF = plotDF[plotDF.Genotype != '5-UTR']
    plotDF = plotDF[plotDF.Genotype != 'outside']

    #make fig and ax
    fig=plt.figure(figsize=(60,10))
    ax=plt.gca()
    
    #make bar plot using genotype and count data
    ax.bar(list(plotDF['Genotype']),list(plotDF['count']), color='dodgerblue')
    
    #make string for title
    title="All single variant mutants across SpAP windows 1-13"

    plt.ylabel('count',fontsize=24)
    plt.yticks(np.arange(0, plotDF['count'].max(), 5), fontsize=24)
    plt.xlabel('Residue (SpAP All, residues 2-542)',fontsize=18)
    plt.xticks(ticks = ['M1','G542'],rotation=45, fontsize=24)
    plt.title(title, fontsize=24)
    plt.tight_layout()
    plt.close()
    return fig

## 1. Import genome and amino acid sequence for reference + identify residues within each mutational region

In [62]:
#import genome (a fasta file with header line and seq line)
genome_file=open('/Users/mja/Appel_MutSequencing/snakemake_files/genomes/spap_with_egfp/spap_with_egfp.fa',"r")

#process file into a list of lines
genome_lines=genome_file.readlines()

#pull out sequence line as string
genome_seq=str(genome_lines[1])

#conver string to list for reference
genome_seq_list=list(genome_seq)


#generate a 'genome' listed in codon format
#use this loop to identify absolute base position of first base in start codon for this genome
for idx, val in enumerate(genome_seq_list):
    start_str="ATG"
    
    #create string of base and the following two bases
    codon_str= str(genome_seq_list[idx])+str(genome_seq_list[idx+1])+str(genome_seq_list[idx+2])
    
    #check if codon above matches start codon, if so break loop, this is first ATG
    if start_str == codon_str:
        
        #store position of first base in start codon
        start_pos = idx
        break
    
#select subset of genome file residing in coding region (here, reference genome
#spans 5'UTR, all of SpAP ORF, and some bases beyond SpAP ORF)    
genome_seq_coding_list=genome_seq_list.copy()

#copy from position of first base in start codon to end
genome_seq_coding_list=genome_seq_coding_list[start_pos:len(genome_seq_coding_list)-1]

#convert coding list back to a string for splitting into triplets
genome_seq_coding_str=''.join(genome_seq_coding_list)

#use regex to split string into list of every 3 characters
genome_seq_codon_list=re.findall('...', genome_seq_coding_str)

## Create a reference dictionary containing AA sequence of each mutational region

In [64]:
# create a dictionary containing residue boundaries for each mutational region
#boundaries are defined with reference to ATG = 1

#translate genome (beginning from start codon, as defined above)
protein=get_translation(genome_seq_coding_str)

#create dict to add boundaries according to region number
#boundaries are displayed as a tuple with (leftmost residue #, rightmost residue #)
#all subsequent plate numbers match mutational regions for this experiment, except 14
#to simplify specify boundaries of 14 (identical to 13)

mutationD={1:[2,41],
           2:[42,89],
           3:[90,137],
           4:[138,185],
           5:[186,232],
           6:[233,279],
           7:[280,326],
           8:[311,356],
           9:[357,402],
           10:[403,448],
           11:[449,491],
           12:[492,517],
           13:[518,542],
           14:[518,542]
          }

#iterate through dict to pull out AA seq for each region and store as new item in each entry's list
for i in mutationD:
    
    #get boundaries for region
    left_AA_pos=mutationD[i][0]-1 #subtract 1 from left boundary position for 0 indexed list of protein AAs
    right_AA_pos=mutationD[i][1] #since non-inclusive, right pos will reference one AA less, don't change
    
    #pull out region within full list of protein AAs, concatenate to a string
    region_list=protein[left_AA_pos:right_AA_pos]   
    mut_region_seq = ''.join(region_list) 
    
    
    #add this string to dict for each entry
    mutationD[i].append(mut_region_seq)
mutationD

{1: [2, 41, 'QSPAPAAAPAPAARSIAATPPKLIVAISVDQFSADLFSEY'],
 2: [42, 89, 'RQYYTGGLKRLTSEGAVFPRGYQSHAATETCPGHSTILTGSRPSRTGI'],
 3: [90, 137, 'IANNWFDLDAKREDKNLYCAEDESQPGSSSDKYEASPLHLKVPTLGGR'],
 4: [138, 185, 'MKAANPATRVVSVAGKDRAAIMMGGATADQVWWLGGPQGYVSYKGVAP'],
 5: [186, 232, 'TPLVTQVNQAFAQRLAQPNPGFELPAQCVSKDFPVQAGNRTVGTGRF'],
 6: [233, 279, 'ARDAGDYKGFRISPEQDAMTLAFAAAAIENMQLGKQAQTDIISIGLS'],
 7: [280, 326, 'ATDYVGHTFGTEGTESCIQVDRLDTELGAFFDKLDKDGIDYVVVLTA'],
 8: [311, 356, 'DKLDKDGIDYVVVLTADHGGHDLPERHRMNAMPMEQRVDMALTPKA'],
 9: [357, 402, 'LNATIAEKAGLPGKKVIWSDGPSGDIYYDKGLTAAQRARVETEALK'],
 10: [403, 448, 'YLRAHPQVQTVFTKAEIAATPSPSGPPESWSLIQEARASFYPSRSG'],
 11: [449, 491, 'DLLLLLKPRVMSIPEQAVMGSVATHGSPWDTDRRVPILFWRKG'],
 12: [492, 517, 'MQHFEQPLGVETVDILPSLAALIKLP'],
 13: [518, 542, 'VPKDQIDGRCLDLVAGKDDSCAGQG'],
 14: [518, 542, 'VPKDQIDGRCLDLVAGKDDSCAGQG']}

In [84]:
combined=pd.DataFrame()

In [85]:
combined_plate_summary_df=pd.DataFrame()

## 2. Import csvs from mutant calling notebook

In [86]:
#specify window and plate to process
expt_windowNumber=1
expt_plateNumber=1

In [87]:
#import initial dataframe created from parsed .vcf file dictionary
#this DF contrains columns (refSeq 	altSeq 	filter 	fw_WT 	rv_WT 	fw_alt 	rv_alt 	position 	i7_barcode)
#and is dual indexed by 'indel_flag' and 'codon'
#it contains all substitutions from all barcodes with variants

#specify plate number to analyze
# plate_num=1

#or, specify list of plate numbers
# plate_num_list=range(1,15)

#specify location of path containing plate-specific mutant calling CSVs
data_path='/Users/mja/Appel_MutSequencing/2021_mut_seq_workup_II/analyses_from_py_notebooks/with_unpaired/mutant_calling/csv_files/plate-well/'
plate_path=data_path+"plate_"+str(expt_plateNumber)+"_"
dataDF=pd.read_csv(plate_path+'mutant_calling_initial_dataframe_all_vars.csv')

#import indel containing dataframe
indels=pd.read_csv(plate_path+'mutant_calling_indels.csv')

#import single base subs with amino acid changes added (REMOVE MULTIPLE POSITIONS PER CODON SWAPPED PER BARCODE)
df_with_subs_culled_BC_C_index=pd.read_csv(plate_path+'mutant_calling_all_AA_subs.csv')

list_of_summary_DFs=[dataDF,indels,df_with_subs_culled_BC_C_index]

## 3. Generate new selected dataframes for plotting

In [100]:
#select subset of columns from imported dataframe containing codon sub information (only 1 pos. included per codon
#per barcode)
single_subs_minimal=df_with_subs_culled_BC_C_index[['i7_barcode',
                                                    'codon',
                                                    'amino_acid_sub',
                                                    'filter',
                                                    'fw_WT',
                                                    'fw_alt',
                                                    'ratio_var',
                                                    'plate',
                                                    'well'
                                                    ]].copy()

single_subs_minimal.set_index('i7_barcode', inplace=True)

#replace 'none'with -1 and NaN with 0 
single_subs_minimal.replace('none',-1, inplace=True)
single_subs_minimal.replace('NaN',0, inplace=True)

#change data type of codon table to int
single_subs_minimal['codon'] = single_subs_minimal['codon'].astype(np.int)

In [ ]:
# df_with_subs_culled_BC_C_index=df_with_subs_culled_BC_C_index.loc[df_with_subs_culled_BC_C_index['ratio_var']>100]

## 4. Create dataframe showing only barcodes with 1 variant

In [102]:
#now plot only barcodes containing a singlemutant
#first, edit dataframe to contain only barcodes that appear once (and with codon substitutions within
#range of ORF)
single_subs_minimal_reset=single_subs_minimal.reset_index()
# subs_indels_empty_reset

single_mutantsDF=single_subs_minimal_reset.groupby('i7_barcode').filter(lambda x: len(x) < 2)

In [95]:
print(single_mutantsDF['amino_acid_sub'].nunique())
print(single_mutantsDF['amino_acid_sub'].unique())

40
['I_28_V' 'R_15_V' 'A_8_V' 'K_23_V' 'A_35_V' 'A_9_V' 'P_22_V' 'Q_2_V'
 'V_30_A' 'P_12_V' 'Q_32_V' 'A_18_V' 'P_10_V' 'L_24_V' 'P_6_V' 'I_17_V'
 'A_11_V' 'F_38_V' 'E_40_V' 'S_16_V' 'P_4_V' 'S_3_V' 'A_5_V' 'A_7_V'
 'P_21_V' 'L_37_V' 'A_14_V' 'I_25_V' 'T_20_V' 'S_39_V' 'A_27_V' 'S_29_V'
 'Y_41_V' 'D_31_V' 'G_656_S' 'A_13_V' 'F_33_V' 'V_26_A' 'S_39_F' 'R_137_L']


## Create dataframe showing only barcodes with 1 variantnuniqued meeting minimum coverage threshold

In [236]:
#now plot only barcodes containing a singlemutant
#first, edit dataframe to contain only barcodes that appear once (and with codon substitutions within
#range of ORF)
subs_indels_empty_min_cov_reset=subs_indels_empty_min_cov.reset_index()
# subs_indels_empty_reset

subs_indels_empty_min_cov_reset_single_var_BCs_only=subs_indels_empty_min_cov_reset.groupby('i7_barcode').filter(lambda x: len(x) < 2)
subs_indels_empty_min_cov_reset_single_var_BCs_only['plate_num']=expt_plateNumber

/home/mason/.conda/envs/orthologs/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [237]:
combined=pd.concat([combined, subs_indels_empty_min_cov_reset_single_var_BCs_only])

len(combined)

2628

In [246]:
combined.to_csv('/media/mason/af107e63-c4f6-4771-8333-2564012a8270/Appel_MutSequencing/mutant_calling/mutant_calling_csvs_FILTER>100/20200121_all_plates_singles_min_cov_only_for_plotting_plate_num.csv')

## 4.1 Plotting histograms per window

In [239]:
# Hist 1.1: plot frequency of variants, only show AA subs within desired mutational window 
#(no coverage theshold)
df_1_1=subs_indels_empty.copy()

# Hist 1.2: plot frequency of variants, only show AA subs within desired mutational window 
#(only mutants above read threshold of >95% of positions in genome having at least 1 read
df_1_2=subs_indels_empty_min_cov.copy()

# Hist 1.3: plot frequency of variants, only show AA subs within desired mutational window 
#(no coverage theshold) [Barcodes with single mutants only]
df_1_3=subs_indels_empty_single_var_BCs_only.copy()

# Hist 1.4: plot frequency of variants, only show AA subs within desired mutational window 
#(only mutants above read threshold of >95% of positions in genome having at least 1 read 
#[Barcodes with single mutants only]
df_1_4=subs_indels_empty_min_cov_reset_single_var_BCs_only.copy()


#create dict with names (for saved file) and dataframes to process
dictofDFs={"window_view_df1_all_vars":df_1_1,
           "window_view_df2_all_vars_mincov":df_1_2,
           "window_view_df3_singles":df_1_3,
           "window_view_df4_singles_mincov":df_1_4
          }
plotPath=('/media/mason/af107e63-c4f6-4771-8333-2564012a8270/Appel_MutSequencing/mutant_calling/mutant_calling_plots_FILTER>100/bar_plots/')

#make bar plot with counts across window for each subsetted dataframe
for key, val in dictofDFs.items():
    
    #create path string for each plate
    plotPath_plate=plotPath+"plate_"+str(expt_plateNumber)
    
    #if path does not exist, make it
    if os.path.isdir(plotPath_plate) == False:
        os.mkdir(plotPath_plate)
        
    else:
        pass
    
    makeVariantHist(expt_windowNumber,expt_plateNumber,val).savefig(plotPath_plate+"/plate_"+str(expt_plateNumber)+"_window_"+str(expt_windowNumber)+"_"+key+'_filter>100.pdf')
    makeVariantHist(expt_windowNumber,expt_plateNumber,val).savefig(plotPath_plate+"/plate_"+str(expt_plateNumber)+"_window_"+str(expt_windowNumber)+"_"+key+'_filter>100.png')
    
    
    #save to joint path for final single var min cov only   
    if 'df4' in key:
        makeVariantHist(expt_windowNumber,expt_plateNumber,val).savefig(plotPath+"/plate_"+str(expt_plateNumber)+"_window_"+str(expt_windowNumber)+"_"+key+'_filter>100.pdf')
        makeVariantHist(expt_windowNumber,expt_plateNumber,val).savefig(plotPath+"/plate_"+str(expt_plateNumber)+"_window_"+str(expt_windowNumber)+"_"+key+'_filter>100.png')
    else:
        pass

## 4.2 Plotting histograms across entire ORF

In [240]:
# makeVariantHist_full(combined).savefig('/media/mason/af107e63-c4f6-4771-8333-2564012a8270/Appel_MutSequencing/mutant_calling/mutant_calling_plots/bar_plots/20200114_all_plates_singles_min_cov_only.pdf')

# makeVariantHist_full(combined).savefig('/media/mason/af107e63-c4f6-4771-8333-2564012a8270/Appel_MutSequencing/mutant_calling/mutant_calling_plots/bar_plots/20200114_all_plates_singles_min_cov_only.png')



## 5. Organizing summary information for each plate

In [241]:
##Pull out counting dataframes from barcodes with any number of variants and barcodes with only one 
##variant

#store df in new variable to reference later, delete rows for non-window positions
var_count_mut_window_all=makeCountDF(expt_windowNumber,expt_plateNumber,subs_indels_empty_min_cov)

#then drop first 4 rows which list positions not in desired windows
var_count_mut_window_all.drop(var_count_mut_window_all.index[[0,1,2,3]], inplace=True)

#store df4 in new variable to reference later, delete rows for non-window positions
var_count_mut_window_singles=makeCountDF(expt_windowNumber,expt_plateNumber,subs_indels_empty_min_cov_reset_single_var_BCs_only)

#then drop first 4 rows which list positions not in desired windows
var_count_mut_window_singles.drop(var_count_mut_window_singles.index[[0,1,2,3]], inplace=True)

In [242]:
#get total number of barcodes processed 
#find number of unique barcodes in dataframe containg all subs, indels, and empty barcodes
index_list=subs_indels_empty.index
total_BCs_processed=index_list.nunique()

# print("number of unique barcodes= " + str(total_BCs_processed))

#get num of barcodes meeting coverage threshold (95% of positions having >=1 read)
subs_indels_empty_min_cov_set_index=subs_indels_empty_min_cov.copy()
subs_indels_empty_min_cov_set_index.set_index('i7_barcode',inplace=True)
index_list_min_cov=subs_indels_empty_min_cov_set_index.index
total_BCs_min_cov=index_list_min_cov.nunique()

#get number of barcodes containing: indels
list_of_unique_indels_min_cov=[]

for index,val in indels_minimal.iterrows():
    if index in index_list_min_cov:
        list_of_unique_indels_min_cov.append(index)
    else:
        pass
num_unique_BC_indels_min_cov=len(list_of_unique_indels_min_cov)

####putting indels aside, how many WT, single mutants, doubles, triple or more?
##first drop duplicates
#use this line to create a new dataframe column in copied df
#and add frequency of each codon to it
subs_indels_empty_min_cov_count=subs_indels_empty_min_cov.copy()
subs_indels_empty_min_cov_count['i7_barcode_freq'] = subs_indels_empty_min_cov_count.groupby('i7_barcode')['i7_barcode'].transform('count')
subs_indels_empty_min_cov_count_dropped=subs_indels_empty_min_cov_count.copy()
subs_indels_empty_min_cov_count_dropped=subs_indels_empty_min_cov_count_dropped.reset_index().drop_duplicates(subset='i7_barcode', keep='first').set_index('i7_barcode')



##num of all non-indels with minimum coverage (note, some of these will have associated indels as well)
#codon designation for indel is -2
num_non_indels_min_cov=len(subs_indels_empty_min_cov_count_dropped[subs_indels_empty_min_cov_count_dropped.codon != -2])

##Now use this df to calculate number of WT, singles, doubles, triples and more
#codon designation for WT is 0
num_WT_min_cov=len(subs_indels_empty_min_cov_count_dropped[subs_indels_empty_min_cov_count_dropped.codon == 0])

#codon designation for 
num_all_mut_min_cov=len(subs_indels_empty_min_cov_count_dropped[subs_indels_empty_min_cov_count_dropped.codon >= 1])

#get single mutants using this selection format
num_singles_min_cov=len(subs_indels_empty_min_cov_count_dropped[(subs_indels_empty_min_cov_count_dropped.codon != -2) & (subs_indels_empty_min_cov_count_dropped.codon != 0) & (subs_indels_empty_min_cov_count_dropped.i7_barcode_freq == 1)])

#get doubles mutants using this selection format
num_doubles_min_cov=len(subs_indels_empty_min_cov_count_dropped[(subs_indels_empty_min_cov_count_dropped.codon != -2) & (subs_indels_empty_min_cov_count_dropped.codon != 0) & (subs_indels_empty_min_cov_count_dropped.i7_barcode_freq == 2)])

#get triples mutants using this selection format
num_triples_plus_min_cov=len(subs_indels_empty_min_cov_count_dropped[(subs_indels_empty_min_cov_count_dropped.codon != -2) & (subs_indels_empty_min_cov_count_dropped.codon != 0) & (subs_indels_empty_min_cov_count_dropped.i7_barcode_freq >= 3)])

####number of desired positions represented in barcodes with just one variant
##use DF copied from section 1.4 and use selection format to find positions with counts !=0
num_pos_desired_total=len(var_count_mut_window_singles)
#rename count column because cannot reference it without getting confused for a counting method
# var_count_mut_window_singles.columns = ['number']

var_count_mut_window_singles.rename(columns={'count':'number'}, inplace=True)
num_pos_cov_singles=len(var_count_mut_window_singles[var_count_mut_window_singles.number > 0])



#####number of desired positions represented in all barcodes
# var_count_mut_window_all.columns = ['number']

var_count_mut_window_all.rename(columns={'count':'number'}, inplace=True)

num_pos_cov_all=len(var_count_mut_window_all[var_count_mut_window_all.number > 0])



#list of barcodes with single variants, and their codon change

#best single variant (one with most reads/highest coverage)

#list of positions not covered


#list of barcodes from multi mutants containing missing positions

plate_summary_df=pd.DataFrame({'plate_num':[expt_plateNumber],
                                'window_num':[expt_windowNumber],
                                'total_BCs':[384], 
                                'BCs_processed':[total_BCs_processed],
                                'BCs_w_min_cov':[total_BCs_min_cov],
                                'unique_indels_min_cov':[num_unique_BC_indels_min_cov],
                                'Num_of_BCs_nonIndels_min_cov':[num_non_indels_min_cov],
                                'num_WT_min_cov':[num_WT_min_cov],
                                'num_singles_min_cov':[num_singles_min_cov],
                                'num_doubles_min_cov':[num_doubles_min_cov],
                                'num_triples_plus_min_cov':[num_triples_plus_min_cov],
                                'num_pos_desired_total':[num_pos_desired_total],
                                'num_pos_cov_singles':[num_pos_cov_singles],
                                'num_pos_cov_all':[num_pos_cov_all],
                               'ratio_singles_to_all':[round((num_singles_min_cov/total_BCs_min_cov),2)]
                               
                              })

#export summary dataframe to csv
plate_summary_df.to_csv('/media/mason/af107e63-c4f6-4771-8333-2564012a8270/Appel_MutSequencing/mutant_calling/mutant_calling_summary_stats_csvs_FILTER>100/plate_'+str(expt_plateNumber)+'_mutant_stats.csv')
plate_summary_df

,plate_num,window_num,total_BCs,BCs_processed,BCs_w_min_cov,unique_indels_min_cov,Num_of_BCs_nonIndels_min_cov,num_WT_min_cov,num_singles_min_cov,num_doubles_min_cov,num_triples_plus_min_cov,num_pos_desired_total,num_pos_cov_singles,num_pos_cov_all,ratio_singles_to_all
0,14,13,384,346,340,11,340,35,210,81,14,25,24,24,0.62


In [243]:
combined_plate_summary_df=pd.concat([combined_plate_summary_df, plate_summary_df])


In [245]:
combined_plate_summary_df

combined_plate_summary_df.to_csv('/media/mason/af107e63-c4f6-4771-8333-2564012a8270/Appel_MutSequencing/mutant_calling/mutant_calling_summary_stats_csvs_FILTER>100/plates_1-14_mutant_stats.csv')
